### LINES

In [ ]:
# Import neccsessary luibraries and first data

import numpy as np
import matplotlib.pyplot as plt
import math

DPI = 150
RAD_1 = 5

In [ ]:
# Functions 
def approx_points(river_X, river_Y, approx):
    gcd_point_X = int(river_X[0] * approx)
    gcd_point_Y = int(river_Y[0] * approx)
    for point_X, point_Y in zip(river_X, river_Y):
        gcd_point_X = math.gcd(gcd_point_X, int(point_X * approx))
        gcd_point_Y = math.gcd(gcd_point_Y, int(point_Y * approx))
    XY_gcd = math.gcd(gcd_point_X, gcd_point_Y)
    approx_river_X = [int(p * approx) / XY_gcd for p in river_X]
    approx_river_Y = [int(p * approx) / XY_gcd for p in river_Y]
    return approx_river_X, approx_river_Y

In [ ]:
import geojson
with open('./pahma.geojson') as f:
    parsed_geojson = geojson.load(f)
    
selected_feature = 2
features = parsed_geojson['features'][selected_feature]
print(f"Count of features: {len(parsed_geojson['features'])}")
print(f"Selected feature: {selected_feature}")
geometry = features['geometry']
coordinates = geometry['coordinates']
print(f"Count of points: {len(coordinates)}")

river_X = list(zip(*coordinates))[0]
river_Y = list(zip(*coordinates))[1] 

river_X, river_Y = approx_points(river_X, river_Y, 5000)

In [ ]:
f = plt.figure()
f.set_dpi(DPI)

plt.axis('equal')

plt.plot(river_X, river_Y, 'black', linewidth = 1)

circle_1 = plt.Circle((196400, 288080), 5, color='r', fill=False)
 
plt.gca().add_artist(circle_1)

plt.show()